In [4]:
import pandas as pd

# Load the CSV file
data = pd.read_csv('homestead co2.csv')

# Print column names to check for any discrepancies
print(data.columns)

# If the column names include spaces or are different, correct them in the code
data['Date'] = pd.to_datetime(data[' Date'].str.strip())  # Adjust the column name as found
data = data.sort_values('Date')  # Ensure sorting is done after parsing the date

# Convert data to JSON format
json_data = data.to_json(orient='records')
print(json_data)

Index([' Date', 'Total CO2e Emissions'], dtype='object')
[{" Date":" 11\/27\/2018","Total CO2e Emissions":109782.87,"Date":1543276800000},{" Date":" 12\/26\/2018","Total CO2e Emissions":117015.51,"Date":1545782400000},{" Date":" 1\/27\/2019","Total CO2e Emissions":135285.42,"Date":1548547200000},{" Date":" 2\/26\/2019","Total CO2e Emissions":156105.3,"Date":1551139200000},{" Date":" 3\/27\/2019","Total CO2e Emissions":125009.73,"Date":1553644800000},{" Date":" 4\/28\/2019","Total CO2e Emissions":91760.91,"Date":1556409600000},{" Date":" 5\/28\/2019","Total CO2e Emissions":74745.66,"Date":1559001600000},{" Date":" 6\/26\/2019","Total CO2e Emissions":49515.57,"Date":1561507200000},{" Date":" 7\/28\/2019","Total CO2e Emissions":55609.35,"Date":1564272000000},{" Date":" 8\/27\/2019","Total CO2e Emissions":71660.34,"Date":1566864000000},{" Date":" 9\/26\/2019","Total CO2e Emissions":86186.34,"Date":1569456000000},{" Date":" 10\/27\/2019","Total CO2e Emissions":96667.86,"Date":1572134400000}

In [7]:
# Load the entire datasets
homestead_df = pd.read_csv('data_files\Homestead.csv')

# Extract relevant columns

# For Monta Vista, use the provided 'Date' and the total CO2 emissions
monta_vista_extracted = monta_vista_df[[' Date', 'Total CO2 Emissions (Therms + KWH to Co2)']]
monta_vista_extracted.columns = ['Date', 'CO2 Emissions']  # Rename columns for uniformity

# Save the extracted data back to CSV if needed
monta_vista_extracted.to_csv('homestead co2.csv', index=False)

monta_vista_extracted.head()

,Date,CO2 Emissions
0,7/10/2023,27719.01
1,6/8/2023,45704.25
2,5/9/2023,198408.60
3,4/10/2023,525936.51
4,3/9/2023,465579.45


In [6]:
# Load the entire datasets
monta_vista_df = pd.read_csv('data_files\Monta Vista.csv')

# Extract relevant columns

# For Monta Vista, use the provided 'Date' and the total CO2 emissions
monta_vista_extracted = monta_vista_df[[' Date', 'Total CO2 Emissions (Therms + KWH to Co2)']]
monta_vista_extracted.columns = ['Date', 'CO2 Emissions']  # Rename columns for uniformity

# Save the extracted data back to CSV if needed
monta_vista_extracted.to_csv('monta vista co2.csv', index=False)

monta_vista_extracted.head()

,Date,CO2 Emissions
0,7/10/2023,27719.01
1,6/8/2023,45704.25
2,5/9/2023,198408.60
3,4/10/2023,525936.51
4,3/9/2023,465579.45


In [15]:
import pandas as pd
import json
import os

# Define the file names
file_names = ["Cupertino.csv", "District Office.csv", "Fremont.csv", "Homestead.csv", "Lynbrook.csv", "Monta Vista.csv"]

# Directory paths
input_directory = '/mnt/data/'
output_directory = '/mnt/data/processed/'

# Create output directory if not exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Function to process CSV files
def process_csv(file_name):
    try:
        # Read the CSV file
        df = pd.read_csv(input_directory + file_name)
        
        # Print columns for debugging
        print(f"Processing {file_name}")
        print(f"Columns: {df.columns.tolist()}")
        
        # Extract Date and CO2 columns
        if file_name == "District Office.csv":
            co2_column = 'Total'
        else:
            co2_column = [col for col in df.columns if 'CO2' in col or 'Carbon' in col]
            if not co2_column:
                print(f"No CO2/Carbon column found in {file_name}")
                return
            co2_column = co2_column[0]
        
        df_co2 = df[[' Date', co2_column]].copy()
        
        # Rename columns
        df_co2.columns = ['Date', 'CO2']
        
        # Save to new CSV
        output_csv_file = output_directory + file_name.replace('.csv', '_co2.csv')
        df_co2.to_csv(output_csv_file, index=False)
        
        # Convert to JSON
        json_data = df_co2.to_json(orient='records')
        output_json_file = output_csv_file.replace('.csv', '.json')
        with open(output_json_file, 'w') as json_file:
            json_file.write(json_data)
        
        print(f"Processed {file_name} successfully.")
    
    except Exception as e:
        print(f"Error processing {file_name}: {e}")

# Process each file
for file_name in file_names:
    process_csv(file_name)


Processing Cupertino.csv
Columns: ['School', ' Date', ' KWH', ' Therms', 'Date Set 2', 'KWH', 'Therms', 'Date Set 3', 'Kwh', 'Therms.1', 'Unnamed: 10', 'KWH Co2', 'Therm CO2', 'Total']
Error processing Cupertino.csv: "[' Date Set 1'] not in index"
Processing District Office.csv
Columns: ['School', ' Date', ' KWH', ' Therms', 'Date Set 2', 'KWH', 'Therm', 'Date Set 3', 'KWH.1', 'Therms', 'KWH Co2', 'Therms Co2', 'Total ']
No CO2/Carbon column found in District Office.csv
Processing Fremont.csv
Columns: ['School', ' Date', ' KWH', ' Therms', 'Date Set 2', 'Unnamed: 5', 'Unnamed: 6', 'Date Set 3', 'Unnamed: 8', 'Unnamed: 9', 'Date Set 4', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'KWH Co2', 'Therms CO2', 'Total']
Error processing Fremont.csv: "[' Date Set 1'] not in index"
Processing Homestead.csv
Columns: ['School', ' Date Set 1', ' KWH', ' Therms', 'Date Set 2', 'KWH #2', 'Therms #2', 'Date Set 3', 'Kwh #3', 'Therms #3', 'KWH TO CO2', 'Therms to CO2', 'Unnamed: 12', 'KWH CO2', 'Therm